<a href="https://colab.research.google.com/github/Center-for-Atmospheric-Research-ATMOS/deposition-calculator/blob/main/deposition_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Here you can estimate the coverage of your sample in the electrostatic precipitator.

In [2]:
#@markdown # Imports
# interaction
from google.colab import files

# calculations
import numpy as np
import pandas as pd
import random
from scipy.integrate import simps
from scipy.optimize import curve_fit
import scipy.stats as stats

# work with text
import re

# visualisation
import plotly.express as px

In [3]:
#@markdown # Upload file
#@markdown Run the cell to upload particle size distribution from AIM in *.TXT
#@markdown format.

datafile = files.upload();

Saving SuccAcidNaCl_2_1_Sample1.txt to SuccAcidNaCl_2_1_Sample1.txt


# Check particle size distribution

In [4]:
#@markdown Function to read file
def read_file(filename):
    '''Retrieves info from the AIM txt file.
    in the returned dataframe diameter is [nm], counts are [particles/cm^3]

    Args:
    filename: last file uploaded to google drive

    Returns:
    df - pd.dataframe containing particle size distribution
    '''
    # read file
    filename = list(datafile.keys())[0]
    with open(filename, 'r', errors='ignore') as f:
        text = f.read()

    # get bins from AIM
    match = re.search(
        r'(?<=Diameter Midpoint\n)(.|\n)*?(?=\nScan Up Time\(s\))',
        text)
    match = match.group().strip()

    # convert bins to float, add bins to df
    df = pd.DataFrame(
        [i.split('\t') for i in match.split('\n')],
        columns=['Diameter', 'Midpoint']).astype('float').dropna()

    # rename columns
    df.columns = ["Diameter", "Counts"]
    return df

In [5]:
#@markdown Function describing lognormal probability density
def lognormal_pdf(x, mu, sigma, scale):
    """Calculates the lognormal probability density function.

    Args:
    x: A NumPy array of particle diameters.
    mu: The mean of the lognormal distribution.
    sigma: The standard deviation of the lognormal distribution.
    scale: The factor that affecting the amplitude the distribution

    Returns:
    A NumPy array of the lognormal probability density function.
    """
    return  scale / (x * sigma * np.sqrt(2 * np.pi)) * np.exp(-(np.log10(x) - mu)**2 / (2 * sigma**2))

In [8]:
# read file
filename = list(datafile.keys())[0]
size_distribution = read_file(filename)

# get diameter to make code clearer
Dp = np.array(size_distribution["Diameter"])

# Calculate the mean logarithmic width of each size bin
dlogDp = np.mean(np.log10(Dp[1:]) - np.log10(Dp[:-1]))

# Multiply the dN/dlog(Dp) values by the logarithmic width to get dN
dN_raw = size_distribution["Counts"] * dlogDp

# plot figure
fig = px.scatter(x=Dp, y=dN_raw, log_x=True)

fig.update_layout(
    xaxis_title="Dp [nm]",
    yaxis_title="dN [particles/cm^3]",
    title="Particle size distribution",
    width=800,
    height=400
)

fig.show()

Here we see that we have several peaks instead of expected one with lognormal distribution.
Presumably this is because the concentration limit of CPC was hit.
Before we proceed the data must be fitted.
For that we will exclude bad points and fit the rest with one peak lognormal probability density function.


In [15]:
# Dp-range to exclude. Data taken from the figure above.
exclude_indices = (Dp >=  68.5) & (Dp <= 461.4)

# initital guess for fitting
# change if the fit does not converge
initial_guess = [0, 1, max(dN_raw)]

# fit a lognormal distribution
params, covariance = curve_fit(
    lognormal_pdf,
    Dp[~exclude_indices],
    dN_raw[~exclude_indices],
    p0=initial_guess)

# extract best fitting parameters
mu, sigma, scale = params

# get final dN in [particles/cm^3], add raw dN
size_distribution["dN_raw"] = dN_raw
size_distribution["dN"] = lognormal_pdf(Dp, mu, sigma, scale)

# plot result
fig = px.scatter(size_distribution,
                 x="Diameter",
                 y=["dN_raw", "dN"],
                 log_x=True)

fig.update_layout(
    xaxis_title="Dp [nm]",
    yaxis_title="dN [particles/cm^3]",
    title="Particle size distribution",
    width=800,
    height=400)

fig.show()

# Theory and methods

The electrical mobility of particles in general is:

$$Z_p = \frac{neC_c(d_p)}{3 \pi \eta d_p}$$

where $n$ is the number of elementary charges $e$ on the particle, $\eta$ is the viscosity of the medium, and $d_p$ is the particle diameter.
$C_c$ is the Cunningham slip correction which is a correction to the friction for particles between the continuum and free molecular regime:

$$C_c = 1 + \frac{2 \lambda}{d_p}(1.142 + 0.558e^{-\frac{0.999d_p}{2 \lambda}})$$

where $\lambda$ is the mean free path.

The particle charge $q=ne$ here is unknown.
But there is a way to restore $Z_p$ from the DMA measurements.

$$Z_p = \frac{q_c + q_m}{4 \pi \Lambda V}$$

where $q_c$ is the flow of sneath air at the DMA entry and $q_m$ the steath air flow at the DMA exit, $V$ is the voltage between inner and outer rods [Knutson and Whitby, 1975].
$\Lambda$ is an instrumental constant given by:

$$\Lambda = \frac{L}{ln(\frac{r_i}{r_o})}$$

where $L$ is the DMA length, $r_i$ and $r_o$ the radii of the inner and outer electrods.

Equating the two ways of calculating $Z_p$, we can derive particle charge:

$$q = \frac{3}{4} \frac{(q_c + q_m) \eta d_p}{\Lambda V C_c(d_p)}$$

The collection efficiency of the precipitator under ideal conditions is described by Deutsch-Anderson equation.

$$eff = 1 - exp(-w\frac{A}{Q})$$

where $A$ is the collecting area of the precipitator, $Q$ is the gas flow through the precipitator.
Migration velocity $w$ is the velocity at which the particle migrates toward the collection electrode within the electrostatic precipitator:

$$w = \frac{qE_p}{3 \pi \eta d_p}$$

The Deutsch-Anderson equation is extensevely used since 1924.
But despite it scientific corectness, it neglects several important factors, such as dust reentrainment, particle size distribution, gas flow variation, and particle sneakage.
Therefore, it should only be used for making preliminary estimates of collection efficiency.
Here, the particle size distribution will be taken into account, and the collection efficiency will be calculated for each particle size.

The particle distribution $c_p(d_p)$ in AIM file is in [particles/cm$^3$].
Concentration of particles at the deposition spot:

$$c_{spot}(d_p) = c_p(d_p) \cdot eff \cdot w \cdot t $$

where $t$ is the deposition time.

Simplified equation from Preger 2020:

$$c_{spot}(d_p) = c_p(d_p) \cdot Z_p \cdot E_p \cdot t $$

To calculate total coverage, we need to divide total area covered by particles by sample area:

$$coverage = \frac{\sum c_{spot}(d_p) \cdot \pi d_p^2}{sample\_area}$$


In [17]:
#@markdown # Retrieve metadata from the file
#@markdown Constants may be input manualy below

# read file
with open(filename, 'r', errors='ignore') as f:
        text = f.read()

# inner DMA radius [m]
ri = float(
    re.findall(
        r'DMA Inner Radius\(cm\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])*1e-2

# outer DMA radius [m]
ro = float(
    re.findall(
        r'DMA Outer Radius\(cm\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])*1e-2

# dinamic size viscosity [Pa*s]
eta = float(
    re.findall(
        r'Reference Gas Viscosity \(Pa\*s\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])

# mean free path [m]
MFP = float(
    re.findall(
        r'Reference Mean Free Path \(m\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])

# sneath flow at the DMA entrance [m^3/s]
qc = float(
    re.findall(
        r'Sheath Flow\(lpm\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0]) * 1e-3 / 60

# sneath flow at the DMA output [lpm]
qm = qc

# DMA characteristic length [m]
L = float(
    re.findall(
        r'DMA Characteristic Length\(cm\)\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])*1e-2

# low voltage [V]
Vmin = float(
    re.findall(
        r'Low Voltage\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])

# high voltage [V]
Vmax = float(
    re.findall(
        r'High Voltage\t[+-]?(\d*\.\d+(?:e[-+]?\d+)?|\d+)',
        text)[0])

print(f'''
ri = {ri} [m]
ro = {ro:.2e} [m]
eta = {eta} [Pa*s]
MFP = {MFP} [m]
qc = {qc} [m^3/s]
qm = {qm} [m^3/s]
L = {L} [m]
Vmin = {Vmin} [V]
Vmax = {Vmax} [V]
''')


ri = 9.37e-05 [m]
ro = 1.96e-04 [m]
eta = 1.83245e-05 [Pa*s]
MFP = 6.73e-08 [m]
qc = 5e-05 [m^3/s]
qm = 5e-05 [m^3/s]
L = 0.00935 [m]
Vmin = 10.2653 [V]
Vmax = 6922.14 [V]



In [18]:
## set constants unavailable in the file
DEPOSITION_TIME = 21*60 # [s], exposure time

PLATE_DIAMETER = 1e-2 # [m], diameter of the precipitator electrode
PRECIPITATOR_D = 0.083 # [m], precipitator diameter
PRECIPITATOR_FLOW = 0.3*1e-3/60 # [m^3/s]

SAMPLE_AREA = .25*1e-4 # [m^2], sample area
Vplate = 1e4 # [V], voltage at the precipitator plate

## calculated constants
# instrumental constant [cm]
Lambda = L / np.log(ri/ro)

# electric field strength at the precipitator plate
Ep = Vplate / (PRECIPITATOR_D - PLATE_DIAMETER) * 2

# precipitator area
Ap = np.pi * PRECIPITATOR_D ** 2

In [22]:
#@markdown # Calculate coverage
# diameter is converted from [nm] to [m]
# dN is converted from [#/cm^3] to [#/m^3]

# add Cunningham slip correction
size_distribution['Cc'] = size_distribution['Diameter'].apply(
    lambda x: 1 + MFP/(x*1e-9) * (2.514 + 0.8*np.exp(-0.55*(x*1e-9)/MFP)))

# add electrical mobility, assume one charge
size_distribution['Zp'] = size_distribution.apply(
    lambda df: 1.6e-19 * 1 * df.Cc / (3 * np.pi * eta * df.Diameter * 1e-9),
    axis=1)

# add drift velocity
size_distribution['Drift_velocity'] = size_distribution.apply(
    lambda df: df.Zp * Ep,
    axis=1)

# add concentration at the spot after deposition for each particle size
size_distribution['Cspot'] = size_distribution.apply(
    lambda df: df.dN * 1e6 * df.Drift_velocity * DEPOSITION_TIME,
    axis=1)

# add total covered area for each particle size
size_distribution['Covered_area'] = size_distribution.apply(
    lambda df: df.Cspot * SAMPLE_AREA * np.pi * (df.Diameter * 1e-9)**2,
    axis=1)

# calculate total coverage
coverage = size_distribution.Covered_area.sum() / SAMPLE_AREA
print(f'Total coverage is {coverage*100:.2f}%')

# check
size_distribution

Total coverage is 53.30%


,Diameter,Counts,dN_raw,dN,Cc,Zp,Drift_velocity,Cspot,Covered_area
0,20.9,6519.30,101.874947,1.697976,11.266923,4.994313e-07,0.136830,2.927420e+08,1.004309e-11
1,21.7,6854.72,107.116444,2.319810,10.874789,4.642777e-07,0.127199,3.717987e+08,1.375046e-11
2,22.5,6916.96,108.089048,3.118953,10.510626,4.327756e-07,0.118569,4.659607e+08,1.852696e-11
3,23.3,6614.28,103.359168,4.131981,10.171552,4.044343e-07,0.110804,5.768781e+08,2.459720e-11
4,24.1,6193.98,96.791279,5.400043,9.855068,3.788430e-07,0.103793,7.062105e+08,3.221500e-11
...,...,...,...,...,...,...,...,...,...
103,850.5,82359.40,1287.003130,1337.289429,1.198993,1.306048e-09,0.000358,6.029235e+08,3.425317e-08
104,881.7,69717.70,1089.455461,1123.345236,1.191938,1.252419e-09,0.000343,4.856693e+08,2.965324e-08
105,914.0,55595.20,868.767820,939.523355,1.185145,1.201274e-09,0.000329,3.896076e+08,2.556288e-08
106,947.5,46357.10,724.407084,782.109891,1.178592,1.152394e-09,0.000316,3.111332e+08,2.193788e-08


# References


*   Instruction Manual: Model 3010 Condensation Particle Counter; 2002, TSI Incorporated, St. Paul, MN, USA.
*   Operation and Service Manual: Series 3080 Electrostatic Classifiers; 2006, TSI Incorporated, St. Paul, MN, USA.
*   E. O. Knutson and K. T. Whitby (1975). Aerosol classification by electric mobility: Apparatus, theory, and applications. J. Aerosol Sci. 6: 443-451.
*   K. Willeke and P.A. Baron: Aerosol Measurement, Van Nostrand, 1993

